# Lesson 1: Introduction to FFmpeg


Welcome to the next lesson, where we will learn how to process and transcribe large audio/video files. In the previous lessons, we've focused on general file handling and basic transcribing techniques. Now, it's time to delve into FFmpeg, a powerful tool that helps manage and manipulate multimedia files. FFmpeg's ability to handle a wide range of audio and video formats makes it an essential tool for anyone working with large files. This lesson will bridge what we've learned about transcribing files with real-world applications using FFmpeg.

## What You'll Learn

In this session, you will:

- Understand the role and utility of FFmpeg in audio and video processing.
- Learn how to use FFmpeg to determine file duration and execute commands.
- Explore how FFmpeg integrates with Python scripts to make multimedia operations seamless.

Let's go!

## Understanding Introduction to FFmpeg

FFmpeg is a versatile command-line tool used for processing video and audio files. It's favored for its flexibility in handling various multimedia formats, making it perfect for transcribing large audio files split into manageable pieces.

At its core, FFmpeg can retrieve audio and video properties, convert files between formats, and perform complex editing. In this lesson, we'll specifically look at how `ffprobe`, a component of FFmpeg, can help us fetch the duration of audio files, which is crucial for splitting them into chunks for transcription.

Understanding how to utilize such features allows you to efficiently manage and manipulate large multimedia files, paving the way for effective transcription and processing.

## Using FFmpeg in Python

Let's dissect the Python code to understand how `ffprobe`, a part of the FFmpeg suite, is used to determine the duration of an audio file. Here's the relevant code snippet for clarity:

```python
def get_audio_duration(file_path):
    """Get the duration of an audio file using ffprobe"""

    # Simulating a terminal command:
    # ffprobe -v quiet -show_entries format=duration -of default=noprint_wrappers=1:nokey=1 <file_path>
    cmd = [
        'ffprobe', 
        '-v', 'quiet',
        '-show_entries', 'format=duration',
        '-of', 'default=noprint_wrappers=1:nokey=1',
        file_path
    ]
    try:
        output = subprocess.check_output(cmd, stderr=subprocess.STDOUT)
        return float(output)
    except subprocess.CalledProcessError:
        return None
```

### Breakdown of the `ffprobe` command:

- `'ffprobe'`: This is the tool from the FFmpeg suite that is capable of retrieving detailed information about multimedia files.
- `'-v', 'quiet'`: These options set the logging level to quiet, suppressing all unnecessary informational messages that `ffprobe` might produce.
- `'-show_entries', 'format=duration'`: This instructs `ffprobe` to specifically pull out the duration entry from the format section of the file's metadata, focusing solely on the length of the audio.
- `'-of', 'default=noprint_wrappers=1:nokey=1'`: This format option adjusts how `ffprobe` outputs the information. By setting `noprint_wrappers=1` and `nokey=1`, it strips out any extra formatting information, keys, or headers, providing a clean, plain output of just the duration value.
- `file_path`: This is the path to the audio file whose duration we want to determine.

If you try running this command in the terminal, the output will contain a single float number — the duration of the media file.

### Example in Bash:

```bash
$ ffprobe -v quiet -show_entries format=duration -of default=noprint_wrappers=1:nokey=1 resources/sample_video.mp4
32.576000
```

By using `subprocess.check_output`, we execute this command line instruction within a Python script, capturing the cleaned output directly. The try-except block ensures we gracefully handle any potential errors during the command execution. The result is the audio file's duration as a float, which is crucial for operations like splitting files for transcription.

## Using Third-Party Libraries for FFmpeg

To make your implementation even better and easier to read, you can use third-party libraries for FFmpeg that provide syntactic sugar and an API that's easier to work with. We will not cover these libraries in the context of this course, but if you are curious, feel free to ask me how to install and use one! The most common example of such a library for Python is `ffmpeg-python`.

## Lesson Summary

In this lesson, you gained an understanding of FFmpeg, a versatile tool for processing multimedia files, and its integration with Python. We explored how to use `ffprobe`, a part of the FFmpeg suite, to determine the duration of audio files, which is crucial for effective transcription. By learning to execute FFmpeg commands within a Python script, you can efficiently manage large multimedia files and integrate processing capabilities into your scripts. These skills enhance your ability to automate tasks and handle complex multimedia challenges in real-world applications.

Let's move on to practice now!


## Retrieving Media Duration is Easy with FFmpeg!

Let's see our ffprobe application in action! Once the task loads, you will see an app where you can choose a couple of media files in the dropdown. For each of them you can see the file's duration - that's the result of our get_audio_duration function you see on your screen (our app calls exactly this function!). Take your time to carefully go through the code and see how it all works.

Note: try running transcribe functionality on the longest video - spot the error that it returns: the video is just too long for Whisper API to process. We'll tackle this issue in the next units!


```python
import subprocess
from openai import OpenAI

client = OpenAI()


def transcribe(file_path):
    """
    Transcribe an audio file using OpenAI's Whisper API.
    """
    try:
        with open(file_path, 'rb') as audio_file:
            transcript = client.audio.transcriptions.create(
                model="whisper-1",
                file=audio_file,
                timeout=60
            )
            return transcript.text
    except Exception as e:
        raise Exception(f"Transcription failed: {str(e)}")


def get_audio_duration(file_path):
    """Get the duration of an audio file using ffprobe"""
    cmd = [
        'ffprobe', 
        '-v', 'quiet',
        '-show_entries', 'format=duration',
        '-of', 'default=noprint_wrappers=1:nokey=1',
        file_path
    ]
    try:
        output = subprocess.check_output(cmd, stderr=subprocess.STDOUT)
        return float(output)
    except subprocess.CalledProcessError:
        return None


```

## Adjust ffprobe Implementation

Let's improve our audio duration script by replacing the basic os.system command with the more robust subprocess.check_output. The check_output function is preferred because it:

Provides better error handling through exceptions
Returns the command output directly as a string
Offers more control over input/output streams
Prevents shell injection vulnerabilities
Run the initial code and see how it behaves now.

Your task is to modify the get_audio_duration function to use subprocess.check_output and add proper error handling for cases when the file doesn't exist or when ffprobe fails.


```python
import subprocess
import os


def get_audio_duration(file_path):
    """Get the duration of an audio file using ffprobe"""
    cmd = f'ffprobe -v quiet -show_entries format=duration -of default=noprint_wrappers=1:nokey=1 "{file_path}"'
    output = os.system(cmd)
    return float(output) if output else None

if __name__ == "__main__":
    # Example usage
    test_file = "resources/sample_audio.mp3"
    duration = get_audio_duration(test_file)
    if duration:
        print(f"Duration of {test_file}: {duration:.2f} seconds")
    else:
        print(f"Could not determine duration of {test_file}")


```

To improve the `get_audio_duration` function by replacing the `os.system` command with `subprocess.check_output`, we will also add error handling to manage cases where the file does not exist or when `ffprobe` fails. Below is the modified code:

```python
import subprocess
import os


def get_audio_duration(file_path):
    """Get the duration of an audio file using ffprobe"""
    if not os.path.isfile(file_path):
        print(f"Error: The file '{file_path}' does not exist.")
        return None

    cmd = ['ffprobe', '-v', 'quiet', '-show_entries', 'format=duration', '-of', 'default=noprint_wrappers=1:nokey=1', file_path]
    
    try:
        output = subprocess.check_output(cmd, stderr=subprocess.STDOUT)
        return float(output.strip()) if output else None
    except subprocess.CalledProcessError as e:
        print(f"Error: ffprobe failed with output: {e.output.decode().strip()}")
        return None
    except ValueError:
        print("Error: Could not convert duration to float.")
        return None


if __name__ == "__main__":
    # Example usage
    test_file = "resources/sample_audio.mp3"
    duration = get_audio_duration(test_file)
    if duration:
        print(f"Duration of {test_file}: {duration:.2f} seconds")
    else:
        print(f"Could not determine duration of {test_file}")
```

### Key Changes Made:

1. **File Existence Check**: Before executing the command, the code checks if the file exists using `os.path.isfile(file_path)`. If the file does not exist, it prints an error message and returns `None`.

2. **Using `subprocess.check_output`**: The command is now constructed as a list and passed to `subprocess.check_output`, which captures the output directly.

3. **Error Handling**: 
   - A `try-except` block is used to catch `subprocess.CalledProcessError`, which handles cases where `ffprobe` fails. The error message is printed, and `None` is returned.
   - A `ValueError` exception is caught to handle cases where the output cannot be converted to a float.

4. **Output Stripping**: The output from `ffprobe` is stripped of whitespace before conversion to ensure that any leading or trailing spaces do not cause conversion errors.

This implementation is more robust and secure, providing better error handling and preventing potential shell injection vulnerabilities.



## Extract and Display Duration For All Media Files

Your task is to modify the audio duration script so that it traverses the resources folder (non-recursively) and prints the audio durations for all files within that directory. Ensure you filter for only media files with appropriate extensions like .mp3, .mp4, .wav, etc.

Use subprocess.check_output for retrieving audio duration with ffprobe, and implement error handling for non-existent files or ffprobe failures.

The expected output is something like this:

```sh
Duration of sample_audio.mp3: XXX seconds
Duration of sample_video.mp4: YYY seconds
```


```python
import subprocess
import os


def get_audio_duration(file_path):
    """Get the duration of an audio file using ffprobe"""
    # TODO: Given the media file path, calculate its duration by calling `ffmpeg` on it using `subprocess.check_output`
    pass


def traverse_and_print_durations(directory):
    """Traverse the specified directory and print durations of media files"""

    valid_extensions = ('.mp3', '.mp4', '.wav')
    # TODO: Use `os.listdir(directory)` to get all files in the directory


if __name__ == "__main__":
    traverse_and_print_durations("resources")


```
To modify the audio duration script so that it traverses the `resources` folder and prints the audio durations for all media files, we will implement the `get_audio_duration` function using `subprocess.check_output`. Additionally, we will filter for valid media file extensions and handle errors appropriately. Below is the complete implementation:

```python
import subprocess
import os


def get_audio_duration(file_path):
    """Get the duration of an audio file using ffprobe"""
    if not os.path.isfile(file_path):
        print(f"Error: The file '{file_path}' does not exist.")
        return None

    cmd = ['ffprobe', '-v', 'quiet', '-show_entries', 'format=duration', '-of', 'default=noprint_wrappers=1:nokey=1', file_path]
    
    try:
        output = subprocess.check_output(cmd, stderr=subprocess.STDOUT)
        return float(output.strip()) if output else None
    except subprocess.CalledProcessError as e:
        print(f"Error: ffprobe failed for '{file_path}' with output: {e.output.decode().strip()}")
        return None
    except ValueError:
        print(f"Error: Could not convert duration to float for '{file_path}'.")
        return None


def traverse_and_print_durations(directory):
    """Traverse the specified directory and print durations of media files"""
    valid_extensions = ('.mp3', '.mp4', '.wav')
    
    # Get all files in the directory
    for filename in os.listdir(directory):
        if filename.lower().endswith(valid_extensions):
            file_path = os.path.join(directory, filename)
            duration = get_audio_duration(file_path)
            if duration is not None:
                print(f"Duration of {filename}: {duration:.2f} seconds")
            else:
                print(f"Could not determine duration of {filename}")


if __name__ == "__main__":
    traverse_and_print_durations("resources")
```

### Explanation of the Code:

1. **Function `get_audio_duration`**: 
   - This function checks if the file exists and constructs the command to retrieve the duration using `ffprobe`.
   - It handles errors for non-existent files and `ffprobe` failures, returning `None` if an error occurs.

2. **Function `traverse_and_print_durations`**:
   - This function lists all files in the specified directory and filters them based on valid media file extensions (`.mp3`, `.mp4`, `.wav`).
   - For each valid file, it calls `get_audio_duration` and prints the duration if available.

3. **Main Execution Block**: 
   - The script starts by calling `traverse_and_print_durations` with the `resources` directory.

### Expected Output:
When you run this script, it will print the duration of each media file in the `resources` directory, formatted as follows:

```sh
Duration of sample_audio.mp3: XXX seconds
Duration of sample_video.mp4: YYY seconds
```

This implementation ensures that the script is robust, handles errors gracefully, and only processes valid media files.Executed 1st Code Block
